In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import style

import pandas as pd
import pandas_datareader.data as web

In [ ]:
style.use('ggplot')

start = dt.datetime(2000, 1, 1)
end = dt.datetime.now()

In [ ]:
tesla_df = web.DataReader("TSLA", 'yahoo', start, end)
elastic_df = web.DataReader("ESTC", 'yahoo', start, end)
netflix_df = web.DataReader("NFLX", 'yahoo', start, end)
okta_df = web.DataReader("OKTA", 'yahoo', start, end)
nasdaq_df = web.DataReader("NDAQ", 'yahoo', start, end)

In [ ]:
nasdaq_df.head()

In [ ]:
tesla_df.tail()

In [ ]:
tesla_df.reset_index(inplace=True)
tesla_df.set_index("Date", inplace=True)

In [ ]:
tesla_df.plot()
plt.show()

In [ ]:
tesla_df['Adj Close'].plot()
plt.show()

In [ ]:
tesla_df[['High','Low']]

In [ ]:
# add the median average at 60 times
tesla_df['moving_average'] = tesla_df['Adj Close'].rolling(window=60, min_periods=0).mean()
elastic_df['moving_average'] = elastic_df['Adj Close'].rolling(window=60, min_periods=0).mean()
netflix_df['moving_average'] = netflix_df['Adj Close'].rolling(window=60, min_periods=0).mean()
okta_df['moving_average'] = okta_df['Adj Close'].rolling(window=60, min_periods=0).mean()
nasdaq_df['moving_average'] = nasdaq_df['Adj Close'].rolling(window=60, min_periods=0).mean()


# add the symbol info
tesla_df['symbol'] = "TSLA"
elastic_df['symbol'] = "ESTC"
netflix_df['symbol'] = "NFLX"
okta_df['symbol'] = "OKTA"
nasdaq_df['symbol'] = "NDAQ"


# add name
tesla_df['name'] = "Tesla, Inc."
elastic_df['name'] = "Elastic NV"
netflix_df['name'] = "Netflix, Inc."
okta_df['name'] = "Okta, Inc."
nasdaq_df['name'] = "Nasdaq, Inc."


# add description
# add name
tesla_df['description'] = "Tesla, Inc. (formerly Tesla Motors, Inc.), is an American electric vehicle and clean energy company based in Palo Alto, California."
elastic_df['description'] = "Elasticsearch is a search engine based on the Lucene library. It provides a distributed, multitenant-capable full-text search engine with an HTTP web interface and schema-free JSON documents. Elasticsearch is developed in Java."
netflix_df['description'] = "Netflix, Inc. is an American media-services provider and production company headquartered in Los Gatos, California, founded in 1997 by Reed Hastings and Marc Randolph in Scotts Valley, California."
okta_df['description'] = "Okta, Inc. is a publicly traded identity and access management company based in San Francisco. It provides cloud software that helps companies manage and secure user authentication into modern applications, and for developers to build identity controls into applications, website web services and devices. It was founded in 2009 and had its initial public offering in 2017, being valued at over $6 billion."
nasdaq_df['description'] = "Nasdaq, Inc. is an American multinational financial services corporation that owns and operates (and is listed on) the Nasdaq stock market and eight European Stock Exchanges, including the Armenian Stock Exchange, Copenhagen Stock Exchange, Helsinki Stock Exchange, Iceland Stock Exchange, Riga Stock Exchange, Stockholm Stock Exchange, Tallinn Stock Exchange, and NASDAQ OMX Vilnius. It is headquartered in New York City, and its president and chief executive officer is Adena Friedman."


In [ ]:
elastic_df.head()

In [ ]:
ax1 = plt.subplot2grid((6,1), (0,0), rowspan=5, colspan=1)
ax2 = plt.subplot2grid((6,1), (5,0), rowspan=1, colspan=1, sharex=ax1)

ax1.plot(tesla_df.index, tesla_df['Adj Close'])
ax1.plot(tesla_df.index, tesla_df['moving_average'])
ax2.bar(tesla_df.index, tesla_df['Volume'])

plt.show()

In [ ]:
netflix_df.head()
nasdaq_df.head()

In [ ]:
tesla_df.to_csv("data/tesla_df.csv")
elastic_df.to_csv("data/elastic_df.csv")
netflix_df.to_csv("data/netflix_df.csv")
okta_df.to_csv("data/okta_df.csv")
nasdaq_df.to_csv("data/nasdaq_df.csv")

In [ ]:
print(elastic_df.shape)
print(netflix_df.shape)
print(tesla_df.shape)
print(okta_df.shape)
print(nasdaq_df.shape)

# Transfor data to elastic documents

In [ ]:
import json
import csv
import itertools

In [ ]:
def lower_space(iterator):
    return itertools.chain([next(iterator).replace(" ", "_").lower()], iterator)
# replace " " with _ and lowercase field names

In [ ]:
# set the name of the csv file where to get data and the name for the index
csv_file_h = "data/nasdaq_df.csv"
index_name = "nasdaq_index"

In [ ]:
data_list = []

with open(csv_file_h) as csv_file:
    # replace a blank space with an underscore
    csv_file = lower_space(csv_file)
    
    csvReader = csv.DictReader(csv_file)

    for row in csvReader:
        for item in row:
            if row[item].isdigit():
                row[item] = int(row[item])
             
            if item != 'date' and item != 'description' and item != 'symbol' and item != 'name':
                row[item] = float(row[item])
            
        data_list.append(dict(row))        

# Following option generate a bulk ready json file (skip if not needed)

In [ ]:
my_header = '{"index": {"_index": "' + index_name + '"}}'

with open('data/nasdaq_data.json', 'w') as bulk_file:
    for x in data_list:
        my_dict = {k:v for k, v in x.items()}
        bulk_file.write(my_header + '\n' + json.dumps(my_dict) + '\n')

# Following option insert data directly to elasticsearch
## 1 - create mapping
## 2 - create elasticsearch client
## 3 - create index and upload data in bulk

In [ ]:
ticker_mapping = {
  "mappings": {
    "properties": {
      "date": {
        "type": "date"
      },
      "high": {
        "type": "float"
      },
      "low": {
        "type": "float"
      },
      "open": {
        "type": "float"
      },
      "close": {
        "type": "float"
      },
      "adj_close": {
        "type": "float"
      },
      "volume": {
        "type": "integer"
      },
        "moving_average": {
        "type": "float"
      },
      "description": {
        "type": "text"
      },
      "symbol": {
          "type": "keyword"
      },
      "name": {
          "type": "text"
      }
    }
  }
}

In [ ]:
from elasticsearch import Elasticsearch, helpers

In [ ]:
# send on cloud
#es = Elasticsearch(
#    cloud_id="your_id:alphanumeric_token_provided_by_elastic_cloud",
#    http_auth=("elastic", "your_amazing_password"),
#)

# connection parameters
host = "localhost"
port = 9200
user = ""
pawd = ""
ssl_context = ""

# elasticsearch client
es = Elasticsearch(hosts=[{'host': host, 'port': port}], scheme="http") #ssl_context=ssl_context, http_auth=(user, pawd)

In [ ]:
index_name = "nasdaq_index"

# delete old index if exists
es.indices.delete(index=index_name, ignore_unavailable=True)

# create index with mapping
r = es.indices.create(index=index_name, body=ticker_mapping)
print(r)

## load in elasticsearch (bulk)
helpers.bulk(es, actions=data_list, index=index_name)

# Combine together all csv files

In [ ]:
combined_csv = pd.concat( [ pd.read_csv(f) for f in ["data/tesla_df.csv", "data/elastic_df.csv", "data/netflix_df.csv", "data/okta_df.csv", "data/nasdaq_df.csv"] ] )
combined_csv.to_csv( "data/combined_tickers.csv", index=False )

In [ ]:
data_list = []
csv_file_h = "data/combined_tickers.csv"

with open(csv_file_h) as csv_file:
    # replace a blank space with an underscore
    csv_file = lower_space(csv_file)
    
    csvReader = csv.DictReader(csv_file)

    for row in csvReader:
        for item in row:
            if row[item].isdigit():
                row[item] = int(row[item])
             
            if item != 'date' and item != 'description' and item != 'symbol' and item != 'name':
                row[item] = float(row[item])
            
        data_list.append(dict(row))        

In [ ]:
index_name = "indeces_combined"

# delete old index if exists
es.indices.delete(index=index_name, ignore_unavailable=True)

# create index with mapping
r = es.indices.create(index=index_name, body=ticker_mapping)
print(r)

## load in elasticsearch (bulk)
helpers.bulk(es, actions=data_list, index=index_name)

# Earning Calendar Association (from Yahoo Finance)

### * BMO — before the market opens
### * AMC — after the market closes
### * TAS/TNS — time not specified

In [ ]:
from yahoo_earnings_calendar import YahooEarningsCalendar
from datetime import timedelta
import dateutil.parser

In [ ]:
# just clean the data frame is exists, not really needed
#del earnings_df

# Ticker from which download earnings list
TICKER = 'NFLX'

# next list of earnings (to be released in future dates)
DAYS_EARNINGS_AHEAD = 3650

# setting the dates:
start_date = start # start is set up above on box In[2]
end_date = (end + timedelta(days=DAYS_EARNINGS_AHEAD))

# downloading the earnings calendar
yec = YahooEarningsCalendar()

earnings_list = yec.get_earnings_of(TICKER)
earnings_df = pd.DataFrame(earnings_list)

earnings_df.head()

In [ ]:
earnings_df = earnings_df.drop(columns=["gmtOffsetMilliSeconds"])

print(earnings_df.shape)

In [ ]:
earnings_df.rename(columns={"ticker": "symbol", 
                            "companyshortname": "name", 
                            "startdatetime": "date", 
                            "startdatetimetype": "type",
                            "epsestimate": "estimate",
                            "epsactual": "reported",
                            "epssurprisepct": "surprise",
                            "quoteType": "quote"}, inplace=True)

# convert all NaN to 0.0
earnings_df["estimate"] = earnings_df["estimate"].fillna(0.0)
earnings_df["reported"] = earnings_df["reported"].fillna(0.0)
earnings_df["surprise"] = earnings_df["surprise"].fillna(0.0)

earnings_df.head()

In [ ]:
# reset index
earnings_df.reset_index(drop=True, inplace=True)
earnings_df.set_index("date", inplace=True)

earnings_df.to_csv("data/netflix_earning_index.csv")

In [ ]:
earnings_mapping = {
  "mappings": {
    "properties": {
      "date": {
        "type": "date"
      },
      "type": {
        "type": "keyword"
      },
      "estimate": {
        "type": "float"
      },
      "reported": {
        "type": "float"
      },
      "surprise": {
        "type": "float"
      },
      "quote": {
        "type": "keyword"
      },
      "symbol": {
          "type": "keyword"
      },
      "name": {
          "type": "text"
      }
    }
  }
}

In [ ]:
data_list = []
csv_file_h = "data/netflix_earning_index.csv"

with open(csv_file_h) as csv_file:
    # replace a blank space with an underscore
    csv_file = lower_space(csv_file)
    
    csvReader = csv.DictReader(csv_file)

    for row in csvReader:
        for item in row:
            if item != 'date' and item != 'type' and item != 'symbol' and item != 'name' and item != 'quote':
                row[item] = float(row[item])
            
        data_list.append(dict(row))

In [ ]:
index_name = "netflix_earning_index"

# delete old index if exists
es.indices.delete(index=index_name, ignore_unavailable=True)

# create index with mapping
r = es.indices.create(index=index_name, body=earnings_mapping)
print(r)

## load in elasticsearch (bulk)
helpers.bulk(es, actions=data_list, index=index_name)